In [2]:
import numpy as np
import plotly.graph_objects as go
import math

def create_dynamic_cache(size):
    """Создание динамического кэша заданного размера"""
    cache = np.empty((size, size), dtype=object)
    return cache

def get_cache_indices(point, cache_size):
    """Получение индексов в кэше для точки"""
    i = int(point[0] * (cache_size - 1))
    j = int(point[1] * (cache_size - 1))
    return min(i, cache_size-1), min(j, cache_size-1)

def circumcenter(triangle):
    """Вычисление центра описанной окружности"""
    (x1, y1), (x2, y2), (x3, y3) = triangle
    
    D = 2 * (x1 * (y2 - y3) + x2 * (y3 - y1) + x3 * (y1 - y2))
    if abs(D) < 1e-10:
        return None
    
    Ux = ((x1*x1 + y1*y1) * (y2 - y3) + (x2*x2 + y2*y2) * (y3 - y1) + (x3*x3 + y3*y3) * (y1 - y2)) / D
    Uy = ((x1*x1 + y1*y1) * (x3 - x2) + (x2*x2 + y2*y2) * (x1 - x3) + (x3*x3 + y3*y3) * (x2 - x1)) / D
    
    return (Ux, Uy)

def in_circle(point, triangle):
    """Проверка, лежит ли точка внутри описанной окружности"""
    center = circumcenter(triangle)
    if center is None:
        return False
    
    radius_sq = (triangle[0][0] - center[0])**2 + (triangle[0][1] - center[1])**2
    dist_sq = (point[0] - center[0])**2 + (point[1] - center[1])**2
    
    return dist_sq < radius_sq - 1e-10

def point_in_triangle(point, triangle):
    """Проверка принадлежности точки треугольнику"""
    def sign(p1, p2, p3):
        return (p1[0] - p3[0]) * (p2[1] - p3[1]) - (p2[0] - p3[0]) * (p1[1] - p3[1])
    
    v1, v2, v3 = triangle
    d1 = sign(point, v1, v2)
    d2 = sign(point, v2, v3)
    d3 = sign(point, v3, v1)
    
    has_neg = (d1 < -1e-10)
    has_pos = (d1 > 1e-10)
    
    return not (has_neg and has_pos)

def get_edges(triangle):
    """Получение рёбер треугольника"""
    return [
        (triangle[0], triangle[1]),
        (triangle[1], triangle[2]),
        (triangle[2], triangle[0])
    ]

def update_cache(cache, triangles):
    """Обновление кэша с учётом текущих треугольников"""
    for i in range(cache.shape[0]):
        for j in range(cache.shape[1]):
            for triangle in triangles:
                if point_in_triangle((i / cache.shape[0], j / cache.shape[1]), triangle):
                    cache[i, j] = triangle
                    break

def resize_cache(old_cache, new_size):
    """Увеличение размера кеша"""
    new_cache = create_dynamic_cache(new_size)
    old_size = old_cache.shape[0]
    for i in range(old_size):
        for j in range(old_size):
            new_cache[2*i, 2*j] = old_cache[i, j]
            new_cache[2*i+1, 2*j] = old_cache[i, j]
            new_cache[2*i, 2*j+1] = old_cache[i, j]
            new_cache[2*i+1, 2*j+1] = old_cache[i, j]
    return new_cache

def delaunay_with_dynamic_cache(points):
    """Триангуляция Делоне с динамическим кешированием"""
    N = len(points)
    cache_size = 2  # Начальный размер кеша
    cache = create_dynamic_cache(cache_size)
    
    max_coord = max(np.max(points[:,0]), np.max(points[:,1])) * 3
    super_triangle = ((-max_coord, -max_coord), (max_coord, -max_coord), (0, max_coord))
    triangles = {super_triangle}
    
    update_cache(cache, triangles)
    
    for point in points:
        point_tuple = (point[0], point[1])
        
        # Увеличение кеша, если необходимо
        if len(triangles) > 3 * cache_size**2:
            cache_size *= 2
            cache = resize_cache(cache, cache_size)
        
        i, j = get_cache_indices(point_tuple, cache_size)
        
        containing_triangle = None
        if point_in_triangle(point_tuple, cache[i,j]):
            containing_triangle = cache[i,j]
        else:
            for triangle in triangles:
                if point_in_triangle(point_tuple, triangle):
                    containing_triangle = triangle
                    break
        
        if containing_triangle:
            bad_triangles = {t for t in triangles if in_circle(point_tuple, t)}
            
            boundary = []
            for triangle in bad_triangles:
                for edge in get_edges(triangle):
                    edge_count = sum(1 for t in bad_triangles for e in get_edges(t) 
                                   if e == edge or e == (edge[1], edge[0]))
                    if edge_count == 1:
                        boundary.append(edge)
            
            triangles = triangles - bad_triangles
            
            for edge in boundary:
                new_triangle = (edge[0], edge[1], point_tuple)
                triangles.add(new_triangle)
    
    update_cache(cache, triangles)
    
    result = {t for t in triangles if not any(p in super_triangle for p in t)}
    
    return result

def plot_triangulation(points, triangles):
    fig = go.Figure()
    
    for triangle in triangles:
        x = [triangle[0][0], triangle[1][0], triangle[2][0], triangle[0][0]]
        y = [triangle[0][1], triangle[1][1], triangle[2][1], triangle[0][1]]
        fig.add_trace(go.Scatter(
            x=x,
            y=y,
            mode='lines',
            line=dict(color='blue', width=1),
            showlegend=False
        ))
    
    fig.add_trace(go.Scatter(
        x=points[:,0],
        y=points[:,1],
        mode='markers',
        marker=dict(color='red', size=5),
        name='Points'
    ))
    
    fig.update_layout(
        title='Триангуляция Делоне с динамическим кешированием',
        xaxis_title='X',
        yaxis_title='Y',
        showlegend=False,
        width=800,
        height=800
    )
    fig.show()

# Тестирование
num_points = 1000
np.random.seed(42)
points = np.random.rand(num_points, 2)
triangles = delaunay_with_dynamic_cache(points)
plot_triangulation(points, triangles)